In [1]:
from pyspark import SparkContext, StorageLevel
from pyspark.sql import SparkSession, udf
from pyspark.sql.functions import date_format, coalesce
from pyspark.sql.types import IntegerType
spark = SparkSession.builder.appName("Spark sle_capacitycapacity_classifier_trend_hist_by_ap_1h").getOrCreate()
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
spark.conf.set("cluster_a/spark.cassandra.connection.host", "cassandra-v3-000-production.mist.pvt,cassandra-v3-001-production.mist.pvt,cassandra-v3-002-production.mist.pvt,cassandra-v3-003-production.mist.pvt,cassandra-v3-004-production.mist.pvt,cassandra-v3-005-production.mist.pvt,cassandra-v3-006-production.mist.pvt,cassandra-v3-007-production.mist.pvt,cassandra-v3-008-production.mist.pvt,cassandra-v3-009-production.mist.pvt")
spark.conf.set("cluster_b/spark.cassandra.connection.host", "cassandra-capacity-000-production.mist.pvt,cassandra-capacity-001-production.mist.pvt,cassandra-capacity-002-production.mist.pvt,cassandra-capacity-003-production.mist.pvt,cassandra-capacity-004-production.mist.pvt,cassandra-capacity-005-production.mist.pvt,cassandra-capacity-006-production.mist.pvt,cassandra-capacity-007-production.mist.pvt,cassandra-capacity-008-production.mist.pvt,cassandra-capacity-009-production.mist.pvt")
spark.udf.registerJavaFunction("cmaps_count","com.mist.spark.udf.CmapCount",IntegerType())
load_options_a = {"cluster": "cluster_a", "table": "capacity_classifier_trend_hist_by_ap_1h", "keyspace": "sle_capacity"}
load_options_b = {"cluster": "cluster_b", "table": "capacity_classifier_trend_hist_by_ap_1h", "keyspace": "sle_capacity"}
df_a = spark.read.format("org.apache.spark.sql.cassandra").options(**load_options_a).load()
df_b = spark.read.format("org.apache.spark.sql.cassandra").options(**load_options_b).load()
df_a.createOrReplaceTempView("table_a")
df_b.createOrReplaceTempView("table_b")
sql_cmd = "SELECT a.`ap_mac` as `ap_mac_a`, b.`ap_mac` as `ap_mac_b`,a.`classifier` as `classifier_a`, b.`classifier` as `classifier_b`,a.`fail` as `fail_a`, b.`fail` as `fail_b`,a.`org_id` as `org_id_a`, b.`org_id` as `org_id_b`,a.`rt` as `rt_a`, b.`rt` as `rt_b`,a.`site_id` as `site_id_a`, b.`site_id` as `site_id_b`,a.`success` as `success_a`, b.`success` as `success_b`,a.`val_avg` as `val_avg_a`, b.`val_avg` as `val_avg_b`,a.`val_avg__count` as `val_avg__count_a`, b.`val_avg__count` as `val_avg__count_b`,a.`val_hist` as `val_hist_a`, b.`val_hist` as `val_hist_b`  FROM (SELECT ap_mac,classifier,fail,org_id,rt,site_id,success,val_avg,val_avg__count,val_hist, hash(`classifier`,`fail`,`org_id`,`success`,`val_avg`,`val_avg__count`,`val_hist`) as hash_a  FROM table_a WHERE rt>=to_timestamp('2020-02-29 10:00:00', 'yyyy-MM-dd HH:mm:ss') and  rt<=to_timestamp('2020-02-29 22:00:00', 'yyyy-MM-dd HH:mm:ss')  ) as a FULL OUTER JOIN (SELECT ap_mac,classifier,fail,org_id,rt,site_id,success,val_avg,val_avg__count,val_hist, hash(`classifier`,`fail`,`org_id`,`success`,`val_avg`,`val_avg__count`,`val_hist`) as hash_b  FROM table_b WHERE rt>=to_timestamp('2020-02-29 10:00:00', 'yyyy-MM-dd HH:mm:ss') and  rt<=to_timestamp('2020-02-29 22:00:00', 'yyyy-MM-dd HH:mm:ss')  ) as b  ON   a.ap_mac =  b.ap_mac AND   a.rt =  b.rt AND   a.site_id =  b.site_id WHERE 1=1 AND NOT (a.hash_a = b.hash_b)   OR a.rt is null OR b.rt is null"
mistmatch_result = spark.sql(sql_cmd.strip())
mistmatch_result.createOrReplaceTempView("mistmatch_result")
mistmatch_result.withColumn("dt", date_format(coalesce(mistmatch_result.rt_a,mistmatch_result.rt_b), "yyyyMMddHHmmss")).repartition(1, "dt").write.partitionBy("dt").mode("overwrite").parquet("s3://mist-cassrator-production/sle_capacity.db/capacity_classifier_trend_hist_by_ap_1h")
load_options_records_count = {"cluster": "cluster_b", "table": "records_count", "keyspace": "biloba"}
spark.sql("with m as (select coalesce(rt_a,rt_b) as rt, sum(case when rt_a is null then 0 else 1 end) as mistmatch_a, sum(case when rt_b is null then 0 else 1 end) as mistmatch_b, count(*) mistmatch_total from mistmatch_result group by rt ), a as (select rt, count(*) as record_count_a from table_a group by rt), b as (select rt, count(*) as record_count_b from table_b group by rt) select keyspace_name, table_name, coalesce(t.rt,m.rt) as rt, record_count_a, record_count_b, mistmatch_a, mistmatch_b, mistmatch_total from (select 'sle_capacity' as keyspace_name, 'capacity_classifier_trend_hist_by_ap_1h' as table_name, coalesce(a.rt,b.rt) as rt, coalesce(a.record_count_a,0) as record_count_a, coalesce(b.record_count_b,0) as record_count_b from a full outer join b on a.rt = b.rt) t full outer join m on t.rt = m.rt  ").write.format("org.apache.spark.sql.cassandra").mode("append").options(**load_options_records_count).save()


AnalysisException: 'Can not load class com.mist.spark.udf.CmapCount, please make sure it is on the classpath;'